In [ ]:
import spectre
import spectre.IO.H5 as spectre_h5
import numpy as np
from spectre.Visualization.ReadH5 import to_dataframe
from spectre.Visualization.PlotCce import plot_cce
import matplotlib.pyplot as plt
from pathlib import Path

# Cauchy-Characteristic Extraction

Here we will go over how to obtain, run, and look at output from SpECTRE's CCE module.

Most of the information in this tutorial can also be found on our [Running CCE tutorial](https://spectre-code.org/tutorial_cce.html).

## Obtaining CCE
 By far the easiest way to obtain the CCE module, is by downloading the pre-built binary from the latest [SpECTRE Release](https://github.com/sxs-collaboration/spectre/releases).



In [ ]:
%%bash
rm -rf CceExecutables*
# TODO: Update link
wget https://github.com/sxs-collaboration/spectre/releases/download/v2024.06.05/CceExecutables.tar.xz
tar -xf CceExecutables.tar.xz

This tarball provides:

- the CCE executable `CharacteristicExtract`
- example worldtube data
- example input file
- example output from CCE
- worldtube data format converter
- python script to check example data

Before we do anything else, let's make sure the executable runs correctly and that we get the expected output. This should run for only a couple minutes.

In [ ]:
%%bash
cd CceExecutables/
rm -f CharacteristicExtractReduction.h5
./CharacteristicExtract --input-file CharacteristicExtract.yaml
python CheckCceOutput.py

### SUCCESS: The CCE output is as expected! Yay!

If you get this message, awesome! That means the executable works correctly.

## The input file

Now let's take a look the input file that we just used and go over the settings that were used. This input file can be used right away to produce production-level CCE waveforms. All you need to change is the path to the worldtube data.

Open the `CceExecutables/CharacteristicExtract.yaml` file in a new tab to get some nice syntax highligting and we'll go over the options (i.e. which ones are important to change for every run and which you should probably leave alone)

## The worldtube data

Now let's take a look at what's required in the worldtube data, and how we expect it to be formatted. First there is an "alphabet soup" of Bondi variables that must be in the H5 file. All these variables must be in the spectre H5::Dat format.

In [ ]:
%%bash
cd CceExecutables/
h5ls -r BondiSachsCceR0200.h5

TODO: Do we put equations here to explain what these variables mean? Maybe Keefe is already going to do this?

In [ ]:
worldtube_data = spectre_h5.H5File("CceExecutables/BondiSachsCceR0200.h5", "r")
bondi_J_dat_file = worldtube_data.get_dat("J")
print(f"Dimensions: {bondi_J_dat_file.get_dimensions()}")
print(f"Version: {bondi_J_dat_file.get_version()}")
print(f"Header:\n{bondi_J_dat_file.get_header()}")

# The legend is a bit too big to print, but we can calculate l_max from the size
legend = bondi_J_dat_file.get_legend()
print(legend[1])
l_max = int(np.sqrt((bondi_J_dat_file.get_dimensions()[1] - 1) / 2) - 1)
print(f"L max: {l_max}")

In [ ]:
# Make the dat file into a Pandas DataFrame for easy manipulation
bondi_J_df = to_dataframe(bondi_J_dat_file)

coef_to_plot = "Re(2,2)"


plt.plot(bondi_J_df["Time"], bondi_J_df[coef_to_plot])
plt.xlabel("Time (M)")
plt.title(f"Bondi J {coef_to_plot}")

So this example data isn't that long. It's only to test out that the executable works.

## Real Worldtube Data

Let's take a look at some real worltube data for a BBH simulation run with SpECTRE.

In [ ]:
cce_data_dir = Path("/oscar/data/icerm/knelli/workshop_materials/03_wednesday/cce")
worldtube_r150_file = cce_data_dir / "BondiSachsCceR0150.h5"
worldtube_data = spectre_h5.H5File(worldtube_r150_file, "r")

# Now it's your turn!
# - Open one of the bondi variable dat files from the H5 file
# - Convert it to a pandas DataFrame
# - Choose a coefficient and plot it!

## Running CCE

First, let's edit the input file in the `CceExecutables` directory to point to the full worltube data. This would be the `BoundaryDataFilename` option. Change it to now point towards `/oscar/data/icerm/knelli/workshop_materials/03_wednesday/cce/BondiSachsCceR0150.h5`

Then, in the `03_Wednesday` directory there is a submit script for you to edit. The only thing you should have to add is the actual command that runs the executable.

> Hint: To specify the input file to run with, add `--input-file NameOfInputFile.yaml` when you run the executable

Once you've done this, submit your job!

In [ ]:
%%bash
sbatch SubmitCCE.sh

This run will take several hours to finish, so you'll have to wait a bit to get your own data.

But don't worry! We've already run CCE for you so have a chance to look at the output data.

## Real CCE output data

The CCE output data is located at `/oscar/data/icerm/knelli/workshop_materials/03_wednesday/cce/CharacteristicExtractReduction.h5`.

Our spectre H5 python bindings know how to read the CCE output data

In [ ]:
cce_output_file_name = cce_data_dir / "CharacteristicExtractReduction.h5"
cce_output_file = spectre_h5.H5File(cce_output_file_name, "r")
# The subfile name always has the extraction radius in it
cce_output = cce_output_file.get_cce("SpectreR0150", 20)

print(f"Dimensions: {cce_output.get_dimensions()}")
print(f"Version: {cce_output.get_version()}")
print(f"Header:\n{cce_output.get_header()}")
legend = cce_output.get_legend()
print(f"Legend size: {legend.size()}")

Rather than getting all the data yourself to plot, the spectre CLI and PyBindings provide a function that will give you a rough plot of all the interesting quantities in the CCE output. This includes

- Strain
- News
- All Weyl scalars

On the command line you'd run

```sh
spectre plot cce -h
```

but since we're in a notebook, we can just call the function directly. This won't give you paper-quality plots, but it can be extremely useful to get a quick glimpse of what is happening with your output.

Pick your favorite modes to plot! Which ones from the previous session seem like they'd be interesting?

In [ ]:
# Modes must be in the format: "Real Y_l,m" or "Imag Y_l,m"
modes = []
fig = plot_cce(cce_output_file_name, modes)
plt.show()